# Dependências

In [ ]:
# Install required Google Cloud packages (commented out as these are typically one-time setup commands)
!pip install gcloud
!gcloud auth application-default login

# Import necessary Python libraries
import pandas as pd                # Data manipulation and analysis
import numpy as np                 # Numerical computing
import time                        # Time-related functions
import os                          # Operating system interfaces
import pandas_gbq                  # Pandas integration with BigQuery
from google.cloud import bigquery  # BigQuery client library
import glob                        # File path pattern matching
import openpyxl                    # Excel file handling
import csv                         # CSV file handling
import re                          # Regular expressions

# Note: The actual imports remain exactly as in the original code

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602927 sha256=1601d8c99c66a48cb129d89a20fec7b3fed8ad7b106423ae8ca7df682210d294
  Stored in directory: /root/.cache/pip/wheels/2a/62/75/3d74209bfebb8805823ae74afa28653aa1ea76d8b5a9d741ff
Successfully built gcloud
Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=ABTdmTHkelkespr8uY8MYqUJxwTchE&prompt=consent&token_usage=remote&access_type=offline&code_chal

# Tratamento

In [1]:
# Define the SQL query to be executed in Google BigQuery.
# This query selects all columns from the `CNES_profissionais_saude_ocupacao_publico` table,
# ordering the results by the 'ano' column in descending order.
query = """
  SELECT * FROM `repositoriodedadosgpsp.Datalake.CNES_profissionais_saude_ocupacao_publico` order by ano desc
        """
# Execute the query using pandas_gbq.read_gbq and load the result into a pandas DataFrame called 'df'.
# The 'project_id' specifies the Google Cloud Project to use.
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')

# Create a new column 'tipo_2'.
# It uses np.where to check if the 'tipo_ocupacao' column contains the string 'Assistentes sociais e economistas domésticos'.
# If it does, the value in 'tipo_2' is set to 'Assistentes sociais e economistas domésticos'; otherwise, it retains the original 'tipo_ocupacao' value.
df['tipo_2'] = np.where(df['tipo_ocupacao'].str.contains('Assistentes sociais e economistas domésticos'),'Assistentes sociais e economistas domésticos',df['tipo_ocupacao'])

# Filter the DataFrame 'df'.
# It keeps only the rows where the 'tipo_2' column is exactly 'Assistentes sociais e economistas domésticos'
# AND the 'ano' column is greater than 2017.
df = df[(df['tipo_2']=='Assistentes sociais e economistas domésticos') & (df['ano']>2017)]

# Read population data from a local CSV file named "Censo_previa_pop.csv" into a new DataFrame called 'pop'.
# The separator for the CSV is specified as a comma.
pop = pd.read_csv("Censo_previa_pop.csv", sep=',')

# Rename columns in the 'pop' DataFrame for better clarity and consistency with the 'df' DataFrame.
pop = pop.rename(columns={'CodMun': 'id_municipio', 'Sigla UF': 'UF', 'PopMun': 'POPULAÇÃO', 'Mun': 'NOME DO MUNICÍPIO'})

# Convert the 'id_municipio' column in the 'df' DataFrame to a numeric data type to ensure proper merging later.
df['id_municipio'] = pd.to_numeric(df['id_municipio'])

# Create a pivot table 'x' from the 'df' DataFrame.
# This aggregates (sums) the 'quantidade_vinculos' (number of employment links)
# for each unique combination of 'ano', 'sigla_uf', 'id_municipio', and 'tipo_2'.
x=df.pivot_table(index=['ano','sigla_uf'  ,'id_municipio','tipo_2'], values='quantidade_vinculos',aggfunc=np.sum)
# Convert the pivot table 'x' (which is a pandas Series with a MultiIndex) into a DataFrame 'y'.
y = pd.DataFrame(x)
# Reset the index of DataFrame 'y' to turn the index levels into columns. The result is assigned back to 'df'.
df = y.reset_index()

# Merge the 'df' DataFrame with a selection of columns from the 'pop' DataFrame.
# The merge is performed on the 'id_municipio' column, which exists in both DataFrames.
# The result is stored in a new DataFrame 'df1' and sorted by 'quantidade_vinculos' in descending order.
df1= df.merge(pop[['UF','NOME DO MUNICÍPIO','id_municipio','POPULAÇÃO']], left_on='id_municipio',right_on='id_municipio').sort_values('quantidade_vinculos', ascending=False)

# Calculate the rate of professionals per 1,000 people and create a new column 'taxa'.
# This is done by dividing the 'quantidade_vinculos' by the 'POPULAÇÃO' and multiplying by 1000.
df1['taxa'] = df1['quantidade_vinculos']/df1['POPULAÇÃO']*1000

# Select a subset of columns from 'df1' and rename them to create the final DataFrame structure.
# The result is assigned back to 'df'.
df = df1[['ano', 'sigla_uf', 'id_municipio', 'tipo_2', 'quantidade_vinculos',
        'NOME DO MUNICÍPIO', 'POPULAÇÃO', 'taxa']].rename(columns={'tipo_2':'tipo_ocupacao','UF':'sigla_uf', 'NOME DO MUNICÍPIO':'nome_municipio','POPULAÇÃO':'populacao_domiciliada' })

# Reorder the columns in the final DataFrame 'df' to a specific desired order.
df= df[['ano', 'sigla_uf', 'nome_municipio','id_municipio','populacao_domiciliada', 'tipo_ocupacao',
       'quantidade_vinculos',
       'taxa']]

# Print a concise summary of the DataFrame 'df'.
# This includes the index dtype and columns, non-null values, and memory usage.
df.info()

NameError: name 'pandas_gbq' is not defined

# Upload

In [ ]:
# Import the bigquery library from google.cloud
from google.cloud import bigquery

# Initialize the BigQuery client, specifying the Google Cloud project ID.
# This client object is used to interact with the BigQuery API.
client = bigquery.Client(project='repositoriodedadosgpsp')

# Create a reference to the BigQuery dataset named 'perfil_remuneracao'.
# This does not create the dataset, but points to it.
dataset_ref = client.dataset('perfil_remuneracao')

# Define the schema for the destination BigQuery table.
# The schema is a list of SchemaField objects, where each object defines a column's:
# 1. Name (e.g., 'ano')
# 2. Data type (e.g., 'INTEGER')
# 3. Description (e.g., 'Ano de referencia da informacao')
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referencia da informacao'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
 bigquery.SchemaField('nome_municipio','STRING',description='Nome do município da observação'),
 bigquery.SchemaField('id_municipio','FLOAT',description='Identificador do município pelo IBGE'),
 bigquery.SchemaField('populacao_domiciliada','FLOAT',description='População domiciliada (prévia do Censo 2022)'),
 bigquery.SchemaField('tipo_ocupacao','STRING',description='Qual a ocupação daquele vínculo'),
 bigquery.SchemaField('quantidade_vinculos','INTEGER',description='Quantidade de vinculos'),
 bigquery.SchemaField('taxa','Float',description='Taxa de Assistentes sociais e economistas domésticos por 1000 habitantes')
 ]

# Create a reference to the target table within the dataset specified earlier.
# The table will be named 'CNES_assistentes_sociais_mil_habitantes_v1'.
table_ref = dataset_ref.table('CNES_assistentes_sociais_mil_habitantes_v1') # table name in the format SOURCE_intuitive_data_name

# Configure the load job by creating a LoadJobConfig object.
# Here, we specify the schema that BigQuery should use when creating or appending to the table.
job_config = bigquery.LoadJobConfig(schema=schema)

# Start the job to load data from the pandas DataFrame 'df' into the specified BigQuery table ('table_ref').
# The job is configured with the previously defined 'job_config'.
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)

# Wait for the load job to complete and retrieve the result.
# This line is blocking and will pause the script's execution until the upload is finished.
job.result()